## Start the 3D Visualizer and the droidlet dashboard

In [ ]:
from droidlet.dashboard.o3dviz import O3DViz
# if False, opens a native window for the 3D visualization. 
# If True, opens a web-based viewer for the 3D stuff that is available at port http://localhost:8889
web_streaming = False 
o3dviz = O3DViz(web_streaming)
o3dviz.start()

from droidlet import dashboard
dashboard.start()
# this has to come after the `dashboard.start` function above
from droidlet.event import sio

## Some useful imports

In [ ]:
import math
import time
import cv2
from matplotlib import pyplot as plt
import open3d as o3d
import numpy as np

## Import and connect to the HelloRobotMover

In [ ]:
from droidlet.lowlevel.hello_robot.hello_robot_mover import HelloRobotMover

In [ ]:
mover = HelloRobotMover(ip="100.95.90.42") # ip of the robot


## Get the rgb, depth and globally-registered point-clouds

In [ ]:
rgb_depth = mover.get_rgb_depth()
img = rgb_depth.rgb
plt.imshow(img)

In [ ]:
plt.imshow(rgb_depth.depth)

In [ ]:
def get_open3d_pcd(rgb_depth):
    points, colors = rgb_depth.ptcloud.reshape(-1, 3), rgb_depth.rgb.reshape(-1, 3)
    colors = colors / 255.
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    return pcd


In [ ]:
point_cloud = get_open3d_pcd(rgb_depth)

o3dviz.put('pointcloud', point_cloud)

## Get the SLAM obstacle map and base state in droidlet canonical co-ordinates

In [ ]:
obstacle_map = mover.get_obstacles_in_canonical_coords()
base_state = mover.get_base_pos_in_canonical_coords()

In [ ]:
# Plot them
x, y = [m[0] for m in obstacle_map], [m[1] for m in obstacle_map]


plt.plot(y, x, 'o', markersize=1)
plt.plot(base_state[0], base_state[1], 'ro', markersize=12)

xorigin, yorigin = 0., 0.
newx = (base_state[0] - xorigin) * math.cos(base_state[2] * math.pi / 180)
newy = (base_state[1] - yorigin) * math.sin(base_state[2] * math.pi / 180)
plt.plot([base_state[0], newx], [base_state[1], newy], 'b')

In [ ]:
# you can also plot it into the dashboard
x, y, yaw = base_state
sio.emit("map",
    {"x": x, "y": y, "yaw": yaw, "map": obstacle_map},
)

## Move the robot using it's navigation system

In [ ]:
# Move forward by 1 metre (x, y, yaw)
mover.move_relative([1.0, 0.0, 0.0], blocking=False)

# turn the robot
mover.turn(-math.radians(20))

## Move the robot directly using it's base API

In [ ]:
mover.bot.go_to_relative([1.0, 0.0, 0.0])

## Move it's camera

In [ ]:
# Set the camera's tilt and pan
mover.bot.set_tilt(math.radians(-60) )
mover.bot.set_pan(math.radians(-30) )

## Stop the robot's actions immediately

In [ ]:
mover.stop()

## Access mover's backend services directly via the RPC classes

In [ ]:
# mover.bot
# mover.slam
# mover.nav
# mover.cam